# Penalty Function
By ZincCat

$\min _{\mathbf{x}} \frac{1}{2}\|\mathbf{x}\|_{2}$

s.t. $\mathbf{A x}=\mathbf{b}$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
p = 200
n = 300

In [ ]:
np.random.seed(19890817)
A = np.random.normal(10, 5, (p, n))
b = np.random.normal(10, 5, p)

def linesearch_Armijo(f, x, g, d, alpha=0.4, beta=0.8):
    # backtrack linesearch using Armijo rules
    t = 1.0
    value = f(x)
    while f(x + t*d) > value + alpha*t*np.dot(g, d):
        t *= beta
    return t

In [ ]:
def f(x):
    return np.linalg.norm(x, 2)/2
def f_ab(gamma):
    return lambda x: np.linalg.norm(x, 2)**2/2 + gamma * np.linalg.norm(A@x-b)**2
def g_ab(gamma):
    return lambda x: x + 2 * gamma * A.T@(A@x-b)
def f_bc(gamma):
    return lambda x: np.linalg.norm(x, 2)**2/2 + gamma * np.sum((A@x-b)**4)
def g_bc(gamma):
    return lambda x: x + 4 * gamma * A.T@((A@x-b)**3)
def f_ab2(gamma):
    return lambda x: np.linalg.norm(x, 2)**2/2 + gamma * np.linalg.norm(A@x-b, 1)
def g_ab2(gamma):
    return lambda x: x + gamma * A.T@np.sign(A@x-b)
cons = ({'type': 'eq', 'fun': lambda x: A.dot(x)-b})

In [ ]:
x0 = np.random.rand(n)

In [ ]:
minValue = f(A.T@np.linalg.inv(A@A.T)@b)
print(minValue)

Direct projected gradient, with inexact line search.

In [ ]:
def descent(f, x, grad):
    # 梯度下降函数
    # 输入函数f, 目前x取值, 梯度函数
    # 输出下降后x取值, 步长t
    xn = x.copy()
    g = grad(xn)
    grad_norm = np.linalg.norm(g, 2)
    d = -g/grad_norm
    t = linesearch_Armijo(f, xn, g, d)
    xn += t*d
    return xn, t

absolute value penalty function

In [ ]:
# 绘图
time1 = []  # 记录时间步, 用于绘图
values1 = []  # 记录某一时间步下函数值, 用于绘图
pvalues1 = [] #  记录某一时间步下含惩罚函数值, 用于绘图
Plot = True  # 是否绘图, 请保证此时alpha, beta均为单一取值
timestep = 0

x = x0.copy() #满足约束的初值
gamma = 0

# 用于判定终止
count = 0 
eps = 1e-10
oldvalue = f(x)
maxIter = 20000  # 最大迭代次数

while True:
    value = f(x)
    print("Iteration:", timestep, "Value", value)
    # 用函数值改变量作为终止条件
    if abs(value - oldvalue) < eps:
        count += 1
    else:
        count = 0
    oldvalue = value
    if timestep > maxIter or count >= 5:
        break
    for i in range(20):
        x, t = descent(f_ab(gamma), x, g_ab(gamma))  # 此时使用无穷范数
    print(f_ab(gamma)(x))
    if Plot:
        time1.append(timestep)
        values1.append(value)
        pvalues1.append(f_ab(gamma)(x))
    gamma += 100
    timestep += 1

Courant-Beltrami penalty function

In [ ]:
# 绘图
time2 = []  # 记录时间步, 用于绘图
values2 = []  # 记录某一时间步下函数值, 用于绘图
pvalues2 = [] #  记录某一时间步下含惩罚函数值, 用于绘图
Plot = True  # 是否绘图, 请保证此时alpha, beta均为单一取值
timestep = 0

x = x0.copy() #满足约束的初值
gamma = 0

# 用于判定终止
count = 0 
eps = 1e-10
oldvalue = f(x)
maxIter = 20000  # 最大迭代次数

while True:
    value = f(x)
    print("Iteration:", timestep, "Value", value)
    # 用函数值改变量作为终止条件
    if abs(value - oldvalue) < eps:
        count += 1
    else:
        count = 0
    oldvalue = value
    if timestep > maxIter or count >= 5:
        break
    for i in range(10):
        x, t = descent(f_bc(gamma), x, g_bc(gamma))  # 此时使用无穷范数
    
    if Plot:
        time2.append(timestep)
        values2.append(value)
        pvalues2.append(f_bc(gamma)(x))
    gamma += 100
    timestep += 1

In [ ]:
plt.plot(time1, np.log(values1)-minValue)
plt.plot(time2, np.log(values2)-minValue)
plt.legend(['Absolute', 'Courant-Beltrami'])
plt.xlabel("Iteration number $(k/100)$")
plt.ylabel("$\log (f(\mathbf{x}_{k})-f^{*})$")
plt.savefig('result1gamma100.png', dpi=200)
plt.show()

并不exact的一点佐证

In [ ]:
# 绘图
time3 = []  # 记录时间步, 用于绘图
values3 = []  # 记录某一时间步下函数值, 用于绘图
Plot = True  # 是否绘图, 请保证此时alpha, beta均为单一取值
timestep = 0

x = x0.copy() #满足约束的初值
gamma = 30000000

# 用于判定终止
count = 0 
eps = 1e-13
oldvalue = f(x)
maxIter = 20000  # 最大迭代次数

while True:
    value = f(x)
    print("Iteration:", timestep, "Value", value)
    # 用函数值改变量作为终止条件
    if abs(value - oldvalue) < eps:
        count += 1
    else:
        count = 0
    oldvalue = value
    if Plot:
        time3.append(timestep)
        values3.append(value)
    if timestep > maxIter or count >= 5:
        break
    for i in range(10):
        x, t = descent(f_ab(gamma), x, g_ab(gamma))  # 此时使用无穷范数
    print(f_ab(gamma)(x))
    # gamma += 1
    timestep += 1